In [2]:
import requests
import pg8000

In [3]:

r = requests.get('http://api.geonames.org/search?featureClass=P&country=FR&maxRows=10&orderby=population&type=json&username=yoannbouge'); 

result = r.json()['geonames']

In [4]:
resu = list(map(lambda x: {'geonameId': x['geonameId'], 'name': x[
            'name'], 'geometry': (x['lat']) + ' ' + x['lng']}, result))

In [18]:
conn = pg8000.connect(user = "postgres", host = "localhost", port = 5432,
                              database = "postgres", password = "password")

In [21]:
for city in resu:
    conn.run("INSERT INTO public.villes(geonameid, geom, name) Values({}, {}, '{}');".format(
        city['geonameId'], "ST_GeomFromText('POINT({})')".format(city['geometry']), city['name']))

In [27]:

conn.run("SELECT * from public.villes")


([2988507, 'Paris', '0101000000280AF4893C6D484027C286A757CA0240'],
 [2995469, 'Marseille', '01010000000B46257502A6454067F2CD3637861540'],
 [2996944, 'Lyon', '01010000009D2E8B89CDDF4640DEC83CF207631340'],
 [2972315, 'Toulouse', '0101000000F623456458CD45400C76C3B64519F73F'],
 [2990440, 'Nice', '0101000000126BF12900DA45408D7A884677101D40'],
 [2990969, 'Nantes', '01010000002B8716D9CE9B47400AA2EE0390DAF8BF'],
 [2973783, 'Strasbourg', '01010000007BBDFBE3BD4A48405DBF60376CFB1E40'],
 [2992166, 'Montpellier', '0101000000991249F432CE4540FBCBEEC9C3020F40'],
 [3031582, 'Bordeaux', '01010000000820B589936B4640FA7E6ABC7493E2BF'],
 [2998324, 'Lille', '0101000000B0AC342905514940F29881CAF8770840'])